# Generating integer table for easy comparison

## Generating data
Fewer scenarios, only integers with 3 digits.
<br/> Multi tables (here 6)
<br/> Only questions of complexity 1, 2, 3.
<br/> Pre-computed answers to be inserted in example prompts.

In [2]:
import pandas as pd
import numpy as np
#np.random.seed(42)
# Careful we're transposing the dataframe so output is different as previously
def generate_custom_data(base_values, trend_multipliers):
    data = {}
    metrics = ['units_sold', 'purchasers', 'generated_turnover', 'average_basket']
    
    for country, multipliers in trend_multipliers.items():
        country_data = {}
        for metric in metrics:
            base = base_values[metric]
            multiplier = multipliers.get(metric, 1)
            if metric in ['units_sold', 'purchasers']:
                # Generate integers directly
                country_data[metric] = np.random.randint(int(base * 0.8 * multiplier), int(base * 1.2 * multiplier))
            elif metric == 'generated_turnover':
                # Calculate turnover based on previously generated units_sold
                unit_price = 10
                turnover = country_data['units_sold'] * unit_price
                country_data[metric] = round(np.random.normal(loc=turnover, scale=turnover * 0.05), 2)
            elif metric == 'average_basket':
                # Calculate average basket size based on previously generated values
                if country_data['purchasers'] > 0:
                    avg_basket = country_data['generated_turnover'] / country_data['purchasers']
                    country_data[metric] = round(np.random.normal(loc=avg_basket, scale=avg_basket * 0.1), 2)
        data[country] = country_data

        # Ensuring 'units_sold' is not less than 'purchasers'
        if data[country]['units_sold'] < data[country]['purchasers']:
            data[country]['units_sold'], data[country]['purchasers'] = data[country]['purchasers'], data[country]['units_sold']
            # modifying average basket 
            print("avg_basket BEFORE: ", avg_basket)
            avg_basket = data[country]['generated_turnover'] / data[country]['purchasers']
            print("avg_basket AFTER: ", avg_basket)
            data[country]['average_basket'] = round(np.random.normal(loc=avg_basket, scale=avg_basket * 0.1), 2)

    # Convert the dictionary to a DataFrame and transpose it
    df = pd.DataFrame(data).T

    # Correct data types
    df['units_sold'] = df['units_sold'].astype(int)
    df['purchasers'] = df['purchasers'].astype(int)
    df['generated_turnover'] = df['generated_turnover'].astype(int)
    df['average_basket'] = df['average_basket'].astype(int)
    # Changed 2 lines above to have integers everywhere.

    return df

### TEST 
base_values = {'units_sold': 5000, 'purchasers': 1500, 'generated_turnover': 50000, 'average_basket': 20}
trend_multipliers = {
    'France': {'units_sold': 1.1, 'purchasers': 1.1, 'generated_turnover': 1.1, 'average_basket': 1.1},
    'Spain': {'units_sold': 1.2, 'purchasers': 1.2, 'generated_turnover': 1.2, 'average_basket': 1.2},
    'Italy': {'units_sold': 0.9, 'purchasers': 0.9, 'generated_turnover': 0.9, 'average_basket': 0.9}
}
df = generate_custom_data(base_values, trend_multipliers)
df

,units_sold,purchasers,generated_turnover,average_basket
France,5615,1596,60255,43
Spain,7100,2000,62341,33
Italy,5099,1269,51556,37


In [3]:
def adjust_base_values(digits_units, digits_purchasers):
    return {
        'units_sold': 5 * 10**(digits_units-1),     # on multiplie par 5 pour centrer et éviter d'avoir 1 digit de moins lors de la génération lorsqu'on multiplie par 0.8 par exemple...
        'purchasers': 5 * 10**(digits_purchasers-1),
        'generated_turnover': 50000,
        'average_basket': 20
    }

scenarios = {
    "Scenario1": (3, 3),
    "Scenario2": (3, 3),
    "Scenario3": (3, 3),
    "Scenario4": (3, 3),
    "Scenario5": (3, 3),
    "Scenario6": (3, 3)
}

multiplier_ranges = {
    'units_sold': (0.8, 1.2),
    'purchasers': (0.8, 1.2),
    'generated_turnover': (0.5, 2.0),
    'average_basket': (0.8, 1.5)
}

In [4]:
import os
os.makedirs('data/multiCSV_integers_V3_multitables', exist_ok=True)

for scenario_name, (digits_units, digits_purchasers) in scenarios.items():
    base_values = adjust_base_values(digits_units, digits_purchasers)
    trend_multipliers = {
        'France': {metric: np.random.uniform(*multiplier_ranges[metric]) for metric in ['units_sold', 'purchasers', 'generated_turnover', 'average_basket']},
        'Spain': {metric: np.random.uniform(*multiplier_ranges[metric]) for metric in ['units_sold', 'purchasers', 'generated_turnover', 'average_basket']},
        'Italy': {metric: np.random.uniform(*multiplier_ranges[metric]) for metric in ['units_sold', 'purchasers', 'generated_turnover', 'average_basket']}
    }
    df = generate_custom_data(base_values, trend_multipliers)
    df.to_csv(f'data/multiCSV_integers_V3_multitables/{scenario_name}.csv')

avg_basket BEFORE:  8.42236797274276
avg_basket AFTER:  10.474427966101695
avg_basket BEFORE:  9.325027027027028
avg_basket AFTER:  10.187775590551182
avg_basket BEFORE:  8.508597285067873
avg_basket AFTER:  9.062168674698796
avg_basket BEFORE:  10.585803402646503
avg_basket AFTER:  10.769019230769231
avg_basket BEFORE:  6.651534090909092
avg_basket AFTER:  9.26651715039578
avg_basket BEFORE:  8.448805704099822
avg_basket AFTER:  9.895156576200417
avg_basket BEFORE:  8.102686084142395
avg_basket AFTER:  10.609025423728813


In [5]:
# Directory containing the CSV files
directory = 'data/multiCSV_integers_V3_multitables'
data_frames = {}    # Initialize an empty dictionary to hold the DataFrames

# Iterate over each file in the directory
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)   # Construct the full file path
    df = pd.read_csv(file_path, index_col=0)        # Read the CSV file into a DataFrame, don't need to use with open()
    scenario_name = filename.replace('.csv', '').replace('_', ' ')  # Extract scenario name from the filename, e.g., 'Scenario1.csv' -> 'Scenario 1'
    data_frames[scenario_name] = df     # Add the DataFrame to the dictionary

In [6]:
# Saving to TXT
directoryT = "data/multiTXT_integers_V3_multitables"
os.makedirs(directoryT, exist_ok=True)  # Ensure the directory exists

for key, frame in data_frames.items():
    table_string = frame.to_string(index=True)
    file_path = os.path.join(directoryT, f"{key}.txt")
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(table_string)

## Generating target
Ideas to make the prompt better:

Examples to give:
- Yes/No
- Categorical: Spain
- Ranking: ['France', 'Spain', 'Italy']
- Integer
- Float (rounded to 2 decimal points)

New:
- Negative values (differences / absolute values instead?)
- Percentages (multiply by 100 ?). "When we ask for a share of something, the number is probably between 0 and 1, so we want you to answer as a percentage (meaning multiply that number by 100)

Also look at the way the question / user prompt is phrased: 
"Which country has the highest average basket" has 100% accuracy
BUT "Which country sold the most units" has worse accuracy.... 
Why? Is it the number of digits, or the way the question is phrased?

Maybe should test the prompt: "Which country has the highest amount of units_sold"

Same thing with:
"Do purchasers in Italy spend more than in France?"
vs
"Is the generated turnover higher in Spain than in France?"

In [7]:
prompts = [
    ##### Niveau 1: Récuparation
    ["How many clients are there in France?",1],    # name of variable is different, possible higher complexity
    ["How many purchasers are there in France?",1],
    ["What's the generated turnover of Spain?",1],
    ["What's the average basket in Italy?",1],
    ["How many units are sold in France?",1],
    ["How many units are sold in Spain?",1],
    ["What's the average basket in Spain?",1],
    ["How many columns are there?",1],
    ["How many countries are there?",1],
    ["How many KPIs are there?",1],

    ##### Niveau 2: Comparaison
    ["Which country has the lowest turnover", 2],
    ["Which country has the highest average basket", 2],
    ["Name the country with the highest number of purchasers", 2],
    ["Which country sold the most units?", 2],
    ["Can you rank the countries in descending order per turnover", 2],
    ["Rank the country names in descending order per turnover", 2],
    ["Do purchasers in France spend more than in Italy?", 2],   # changed to get rid of (answer by yes or no)
    ["Do purchasers in Italy spend more than in France?", 2],
    ["Is the generated turnover higher in Spain than in France?", 2],
    ["Did Italy sell more units than Spain?", 2],

    ##### Niveau 3: Opération élémentaire
    ["What is the total generated turnover across all countries?", 3],
    ["What is the total number of purchasers across all countries?", 3],
    ["What is the total number of units sold across all countries?", 3],
    ["What is the average generated turnover across all countries?", 3],        # average is 2 "opération élémentaire", could be considered more complex than sum
    ["What is the average number of purchasers across all countries?", 3],
    ["What is the average basket size across all countries?", 3],
    ["How many more clients are there in Italy compared to France", 3],
    ["What's the absolute difference between number of purchasers in Italy compared to number of purchasers in France?", 3],
    ["Which is the average turnover per client in Italy", 3],
    ["Which is the average turnover per client in France?", 3]
]

In [8]:
examplePrompts = [
    ["What's the average basket in France?",1],    # Niveau 1: Récupération (Integer)
    ["Which country has the highest turnover?",2], # Niveau 2: Comparaison (Category)
    ["List the country names in descending order per number of purchasers?",2], # Niveau 2: Comparaison (List)
    ["Does France have a higher generated turnover than Italy?",2], # Niveau 2: Comparaison (Yes/No)
    ["What is the average number of units sold across all countries?",3],  # Niveau 3: Opération élémentaire (Rounded Float)
]
    # Niveau 3: Opération élémentaire (Percentage share) Possibly
    # Niveau 3: Opération élémentaire (Negative Number) Possibly    

In [9]:
# Final target calculation with reordered prompts

# Initialize the DataFrame for prompts and answers
targetDF = pd.DataFrame(columns=["Prompt", "Complexité"] + [f"Target {scenario}" for scenario in data_frames.keys()])

# Extend the DataFrame for new prompts
for promptL in prompts:
    targetDF.loc[len(targetDF), ['Prompt', 'Complexité']] = [promptL[0], promptL[1]]  # promptL[1] is "Complexité"

# Iterate through all the prompts to populate target columns
for idx, prompt_row in targetDF.iterrows():
    prompt = prompt_row['Prompt']
    for scenario, df_to_measure in data_frames.items():

        # Niveau 1: Récuparation
        if prompt == "How many clients are there in France?":
            value = df_to_measure.at['France', 'purchasers']

        elif prompt == "How many purchasers are there in France?":
            value = df_to_measure.at['France', 'purchasers']

        elif prompt == "What's the generated turnover of Spain?":
            value = df_to_measure.at['Spain', 'generated_turnover']

        elif prompt == "What's the average basket in Italy?":
            value = df_to_measure.at['Italy', 'average_basket']

        elif prompt == "How many units are sold in France?":
            value = df_to_measure.at['France', 'units_sold']

        elif prompt == "How many units are sold in Spain?":
            value = df_to_measure.at['Spain', 'units_sold']

        elif prompt == "What's the average basket in Spain?":
            value = df_to_measure.at['Spain', 'average_basket']

        elif prompt == "How many columns are there?":
            value = df_to_measure.shape[1]  # columns

        elif prompt == "How many countries are there?":
            value = df_to_measure.shape[0]  # Assuming that 'countries' are the rows of the DataFrame

        elif prompt == "How many KPIs are there?":
            value = df_to_measure.shape[1]  # Assuming that 'KPIs' are the columns of the DataFrame
        

        # Niveau 2: Comparaison
        elif prompt == "Which country has the lowest turnover":
            value = df_to_measure['generated_turnover'].idxmin()

        elif prompt == "Which country has the highest average basket":
            value = df_to_measure['average_basket'].idxmax()

        elif prompt == "Name the country with the highest number of purchasers":
            value = df_to_measure['purchasers'].idxmax()

        elif prompt == "Which country sold the most units?":
            value = df_to_measure['units_sold'].idxmax()

        elif prompt == "Can you rank the countries in descending order per turnover":
            value = df_to_measure['generated_turnover'].sort_values(ascending=False).index.tolist()
            value = ', '.join(value)    # Convert the list of countries into a comma-separated string

        elif prompt == "Rank the country names in descending order per turnover":
            value = df_to_measure['generated_turnover'].sort_values(ascending=False).index.tolist()
            value = ', '.join(value)    # Convert the list of countries into a comma-separated string

        elif prompt == "Do purchasers in France spend more than in Italy?":
            turnover_france = df_to_measure.at['France', 'generated_turnover']
            turnover_italy = df_to_measure.at['Italy', 'generated_turnover']
            value = "Yes" if turnover_france > turnover_italy else "No"

        elif prompt == "Do purchasers in Italy spend more than in France?":
            turnover_france = df_to_measure.at['France', 'generated_turnover']
            turnover_italy = df_to_measure.at['Italy', 'generated_turnover']
            value = "Yes" if turnover_france < turnover_italy else "No"

        elif prompt == "Is the generated turnover higher in Spain than in France?":
            turnover_france = df_to_measure.at['France', 'generated_turnover']
            turnover_spain = df_to_measure.at['Spain', 'generated_turnover']
            value = "Yes" if turnover_france < turnover_spain else "No"

        elif prompt == "Did Italy sell more units than Spain?":
            units_italy = df_to_measure.at['Italy', 'units_sold']
            units_spain = df_to_measure.at['Spain', 'units_sold']
            value = "Yes" if units_italy > units_spain else "No"

        # Niveau 3: Opération élémentaire
        elif prompt == "What is the total generated turnover across all countries?":
            value = int(df_to_measure['generated_turnover'].sum())

        elif prompt == "What is the total number of purchasers across all countries?":
            value = int(df_to_measure['purchasers'].sum())

        elif prompt == "What is the total number of units sold across all countries?":
            value = int(df_to_measure['units_sold'].sum())

        elif prompt == "What is the average generated turnover across all countries?":
            value = round(df_to_measure['generated_turnover'].mean(), 2)

        elif prompt == "What is the average number of purchasers across all countries?":
            value = round(df_to_measure['purchasers'].mean(), 2)

        elif prompt == "What is the average basket size across all countries?":
            value = round(df_to_measure['average_basket'].mean(), 2)

        elif prompt == "How many more clients are there in Italy compared to France":
            value = df_to_measure.at['Italy', 'purchasers'] - df_to_measure.at['France', 'purchasers']

        ###################################################
        elif prompt == "What's the absolute difference between number of purchasers in Italy compared to number of purchasers in France?":
            value = abs(df_to_measure.at['Italy', 'purchasers'] - df_to_measure.at['France', 'purchasers'])

        elif prompt == "Which is the average turnover per client in Italy":
            value = round(df_to_measure.at['Italy', 'generated_turnover'] / df_to_measure.at['Italy', 'purchasers'], 2)

        elif prompt == "Which is the average turnover per client in France?":
            value = round(df_to_measure.at['France', 'generated_turnover'] / df_to_measure.at['France', 'purchasers'], 2)

        # Update the DataFrame
        targetDF.at[idx, f"Target {scenario}"] = value

targetDF.set_index('Prompt', inplace=True)
targetDF

,Complexité,Target Scenario1,Target Scenario2,Target Scenario3,Target Scenario4,Target Scenario5,Target Scenario6
Prompt,,,,,,,
How many clients are there in France?,1,472,467,450,379,518,472
How many purchasers are there in France?,1,472,467,450,379,518,472
What's the generated turnover of Spain?,1,5175,5153,6684,5123,4465,5879
What's the average basket in Italy?,1,11,9,9,9,11,11
How many units are sold in France?,1,587,527,659,528,522,618
How many units are sold in Spain?,1,555,538,671,538,429,577
What's the average basket in Spain?,1,9,9,11,14,10,13
How many columns are there?,1,4,4,4,4,4,4
How many countries are there?,1,3,3,3,3,3,3


In [11]:
# Saving target to csv
targetDF.to_csv('target/target_digit_integer_V3_multitables', sep=';', index=True)

In [12]:
########### NEW ! Example target
# Initialize the DataFrame for prompts and answers
exampleTargetDF = pd.DataFrame(columns=["examplePrompt", "Complexité"] + [f"Target {scenario}" for scenario in data_frames.keys()])

# Extend the DataFrame for new prompts
for promptL in examplePrompts:
    exampleTargetDF.loc[len(exampleTargetDF), ['examplePrompt', 'Complexité']] = [promptL[0], promptL[1]]  # promptL[1] is "Complexité"

# Iterate through all the prompts to populate target columns
for idx, prompt_row in exampleTargetDF.iterrows():
    prompt = prompt_row['examplePrompt']
    for scenario, df_to_measure in data_frames.items():

        if prompt == "What's the average basket in France?":     # Niveau 1: Récupération (Integer)
            value = df_to_measure.at['France', 'average_basket']

        elif prompt == "Which country has the highest turnover?": # Niveau 2: Comparaison (Category)
            value = df_to_measure['generated_turnover'].idxmax()

        elif prompt == "List the country names in descending order per number of purchasers?":
            value = df_to_measure['purchasers'].sort_values(ascending=False).index.tolist()
            value = ', '.join(value)    # Convert the list of countries into a comma-separated string

        elif prompt == "Does France have a higher generated turnover than Italy?": # Niveau 2: Comparaison (Yes/No)
            turnover_france = df_to_measure.at['France', 'generated_turnover']
            turnover_italy = df_to_measure.at['Italy', 'generated_turnover']
            value = "Yes" if turnover_france > turnover_italy else "No"

        elif prompt == "What is the average number of units sold across all countries?":    # Niveau 3: Opération élémentaire (Rounded Float)
            value = round(df_to_measure['units_sold'].mean(), 2)

        # Update the DataFrame
        exampleTargetDF.at[idx, f"Target {scenario}"] = value

exampleTargetDF.set_index('examplePrompt', inplace=True)
exampleTargetDF

,Complexité,Target Scenario1,Target Scenario2,Target Scenario3,Target Scenario4,Target Scenario5,Target Scenario6
examplePrompt,,,,,,,
What's the average basket in France?,1,10,9,12,9,10,10
Which country has the highest turnover?,2,Spain,Spain,Spain,Spain,Italy,Spain
List the country names in descending order per number of purchasers?,2,"Spain, France, Italy","Spain, France, Italy","Spain, Italy, France","Italy, Spain, France","France, Italy, Spain","Italy, Spain, France"
Does France have a higher generated turnover than Italy?,2,No,Yes,Yes,No,No,No
What is the average number of units sold across all countries?,3,561.67,502.33,619.67,542.33,503.67,586.67


In [13]:
# Saving target to csv
exampleTargetDF.to_csv('target/exampleTarget_digit_integer_V3_multitables', sep=';', index=True)

Thoughts:

1. The original question: "Rank the country names in descending order per turnover" can be problematic since there is not enough variation (according to me) 3 out of 6 answers are [France, Spain, Italy]...